### 转置卷积的实现

In [ ]:
import tensorflow as tf

# 创建 4×4 大小输入
x = tf.range(16)+1
x = tf.cast(tf.reshape(x, [1, 4, 4, 1]), tf.float32)

# 创建 3×3 卷积核
w = tf.constant([[-1., 2, -3], [4, -5, -6], [-7, 8, -9]])
w = tf.expand_dims(w, axis=2)
w = tf.expand_dims(w, axis=3)
# print(w)


In [ ]:
# 普通卷积运算
out = tf.nn.conv2d(x, w, strides=1, padding='VALID')
# print(out)

In [ ]:
# 保持 strides=1, padding='VALID',卷积核不变的情况下，通过卷积核于输出 out 的转置卷积运算恢复与 x 相同大小的高宽张量
xx = tf.nn.conv2d_transpose(out, w, strides=1, padding='VALID', output_shape=[1, 4, 4, 1])
xx


### 实现 ResBlock

In [ ]:
from tensorflow import keras
from tensorflow.keras import datasets, layers, Sequential, optimizers, losses

基本块单元

In [ ]:
class BasicBlock(layers.Layer):
    def __init__(self, filter_num, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = layers.Conv2D(filter_num, (3, 3), strides=stride, padding='same')
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.Activation('relu')

        self.conv2 = layers.Conv2D(filter_num, (3, 3), strides=1, padding='same')
        self.bn2 = layers.BatchNormalization()

        # 形状的的转换
        # stride > 1,相当于下采样
        if stride != 1:
            self.downsample = Sequential()
            self.downsample.add(layers.Conv2D(filter_num, (1, 1), strides=stride))
        else:
            self.downsample = lambda x:x
    # 前向传播函数，[b, h, w, c]
    def call(self, inputs, training=None):
        out = self.conv1(inputs)
        out = self.bn1(out, training=training)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out, training=training)
        identity = self.downsample(inputs)
        # f(x)+x的运算
        add = layers.add([out, identity])
        # 不建议使用 self.relu()
        output = tf.nn.relu(add)
        return output

单元块组装

In [ ]:
class ResNet(keras.Model):
    # [2,2,2,2],全连接层输出大小 100
    def __init__(self, layer_dims,  num_classes=10):
        super(ResNet, self).__init__()
        # 第一层
        self.stem = Sequential([layers.Conv2D(64, (3, 3), strides=(1, 1)), 
                                layers.BatchNormalization(),
                                layers.Activation('relu'),
                                layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding='same')
                                ])
        # 64, 128等表示通道数
        self.layer1 = self.build_resblock(64,  layer_dims[0])
        self.layer2 = self.build_resblock(128, layer_dims[1], stride=2)
        self.layer3 = self.build_resblock(256, layer_dims[2], stride=2)
        self.layer4 = self.build_resblock(512, layer_dims[3], stride=2)
        
        # [b ,512, h, w]=>[b, 512, 1, 1]
        # 输出层
        self.avgpool = layers.GlobalAveragePooling2D()
        self.fc = layers.Dense(num_classes)
        
    def call(self, inputs, training=None, mask=None):
        x = self.stem(inputs, training=training)
        x = self.layer1(x, training=training)
        x = self.layer2(x, training=training)
        x = self.layer3(x, training=training)
        x = self.layer4(x, training=training)

        x = self.avgpool(x)
        x = self.fc(x)

        return x
    
    def build_resblock(self, filter_num, blocks, stride=1):
        # blocks残差层块数,如blocks=2
        res_block = Sequential()
        res_block.add(BasicBlock(filter_num, stride))

        for _ in range(1, blocks):
            res_block.add(BasicBlock(filter_num, stride=1))
        return res_block       

resnet18 网络模型定义

In [ ]:
def resnet18():
    # layer_dims
    return ResNet([2, 2, 2, 2])

resnet34 网络模型定义

In [ ]:
def resnet34():
    # layer_dims
    return ResNet([3, 4, 6, 3])

### 数据集训练 ResNet18

In [ ]:
# 可选择导入自定义的包 resnet
# from resnet import resnet18

In [ ]:
# 数据的前处理模块
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255. - 1.
    y = tf.cast(y, dtype=tf.int32)
    return x, y

In [ ]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
print(x_train.shape,y_train.shape)
y_train = tf.squeeze(y_train, axis=1)
y_test = tf.squeeze(y_test, axis=1)
train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
train_db = train_db.map(preprocess).shuffle(1000).batch(128)
test_db = test_db.map(preprocess).batch(128)
example = next(iter(train_db))
print("一个batch维度：", example[0].shape, example[1].shape)
print("x最小值：",tf.reduce_min(example[0]), "x的最大值", tf.reduce_max(example[0]))

In [ ]:
model = resnet18()
# 第一层为 3×3,s=1,32+1-3/1=30
model.build(input_shape=[None, 32, 32, 3])
model.summary()

In [ ]:
optimizer = optimizers.Adam(learning_rate=0.001)
for epoch in range(10):
    for step, (x,y) in enumerate(train_db):
        with tf.GradientTape() as tape:
            # [b, 32, 32, 3]=> [b, 100]
            logits = model(x)
            y_onehot = tf.one_hot(y, depth=10)
            loss = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)
            loss = tf.reduce_mean(loss)
        grads = tape.gradient(loss, model.variables)
        optimizer.apply_gradients(zip(grads, model.variables))
        
        if step % 200 == 0:
            print(epoch, step, '损失：', float(loss))


    # 测试
    # total_num, total_correct = 0, 0
    # for x,y in test_db:
    #     logits = model(x, training=False)
    #     prob = tf.nn.softmax(logits, axis=1)
    #     pred = tf.argmax(prob, axis=1)
    #     pred = tf.cast(pred, dtype=tf.int32)

    #     correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
    #     correct = tf.reduce_sum(correct)

    #     total_num += x.shape[0]
    #     total_correct += int(correct)

    # acc = total_correct / total_num
    # print(epoch, 'acc:', acc)